In [ ]:
import torch
a=torch.tensor(11)
a.shape

In [ ]:
weight=torch.ones(16,768,16,16)
weight*torch.tensor(12)

In [ ]:
import torch
Wq=torch.randint(0,255,(64,32,3,3))
# with open ('TxT//ConvWeight.txt','a') as ff:
#     #Wq的维度：[OC,IC,K,K]
#     for OC in range(Wq.shape[0]):#遍历全部输出通道
#         for K_Row in range(Wq.shape[2]):#遍历行
#             for K_Col in range(Wq.shape[3]):#遍历列
#                 for IC in range(0,Wq.shape[1],8):#遍历通道
#                     ff.write('%02x'%(int(Wq[OC,IC:IC+7,K_Row,K_Col].item())&0xff))
#                     ff.write("\n")
#     ff.close()
for OC in range(Wq.shape[0]):#遍历全部输出通道
    for K_Row in range(Wq.shape[2]):#遍历行
        for K_Col in range(Wq.shape[3]):#遍历列
            for IC in range(0,Wq.shape[1],8):#遍历通道
                # print(list(Wq[OC,IC:IC+2,K_Row,K_Col]))
                Hex0=Wq[OC,IC,K_Row,K_Col].item()&0xff
                Hex1=Wq[OC,IC+1,K_Row,K_Col].item()&0xff
                Hex2=Wq[OC,IC+2,K_Row,K_Col].item()&0xff
                Hex3=Wq[OC,IC+3,K_Row,K_Col].item()&0xff
                Hex4=Wq[OC,IC+4,K_Row,K_Col].item()&0xff
                Hex5=Wq[OC,IC+5,K_Row,K_Col].item()&0xff
                Hex6=Wq[OC,IC+6,K_Row,K_Col].item()&0xff
                Hex7=Wq[OC,IC+7,K_Row,K_Col].item()&0xff
                print('%02x%02x%02x%02x%02x%02x%02x%02x'%(Hex7,Hex6,Hex5,Hex4,Hex3,Hex2,Hex1,Hex0))
                # ff.write("\n")    

In [ ]:
import os
filename = r'E:\Transformer\Transformer_Arithmatic\Transformer_Main\TxT\Weight.txt'

# 判断文件是否存在不存在则创建一个
if not os.path.isfile(filename):
     fd = open(filename, mode="w", encoding="utf-8")


In [ ]:
import torch.nn as nn
picture_in=torch.rand(1,3,224,224)

nn.conv(picture_in)

In [ ]:
for i in range(0,32):
    print(i)

In [ ]:
import torch
import torch.nn as nn
import numpy as np
Wq=torch.randint(0,10,(64,32,3,3),dtype=torch.float)
Xq=torch.randint(0,10,(1,32,322,322),dtype=torch.float)
# Xq=torch.rand(1,32,322,322)
ConvLayer=nn.Conv2d(32,64,3,bias=False,stride=2,padding=0)
ConvLayer.weight.data=Wq
Output=ConvLayer(Xq)
print(Output.shape)

In [12]:
import math
import torch
import numpy as np
def gen_B(S1, S2, S3):  # 第一组  β即N_REAL  +++++求shitf++++
    M = (S1 * S2) / S3
    M = M.numpy()

    daxiao = S2.shape[0]  # 第一层权重的shape[0]是32 shape[0]表示行数 是一维大小位32的列向量
    SCALE = np.zeros(daxiao, dtype=np.uint32, order='C')  # 相当于32个输出通道 每个对应一组shift
    N_REAL = np.zeros(daxiao, dtype=np.uint32, order='C')
    for i, ii in enumerate(M):  # enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标

        while not (ii >= 0.5 and ii <= 1.0):  # 左移到（0.5，1） 左移一次相当于*2
            ii *= 2
        pass
        mmmm = ii * (2 ** 32)  # 乘2^32

        SCALE[i] = mmmm.astype(np.int32)#更新Scale放大后的值

    for i, ii in enumerate(M):#存在一个问题，如果shift为0怎么办？
        N_REAL[i] = round(math.log(SCALE[i] / ii, 2)) - 32 - 1  # fpga加了1这里要减1,  β值也是m维 相当于存的mmmm

    return N_REAL
S1=torch.rand(1)
S2=torch.rand(10)
S3=torch.rand(1)
gen_B(S1,S2,S3)

array([ 7,  6,  4,  5,  6,  9,  5, 11,  9,  4], dtype=uint32)

In [ ]:

def gen_M_N(S1, S2, S3):  # 第二组求M'即M  ++++++返回scale和shift+++++
    daxiao = S2.shape[0]
    M = np.zeros(daxiao, dtype=np.uint32, order='C')
    N_REAL = gen_B(S1, S2, S3)
    M = np.zeros(S2.shape[0])
    for i, ii in enumerate(M):
        M[i] = (torch.round((S1 * S2[i]) / S3 * (2 ** (32 + N_REAL[i] + 1)))).numpy()  # s1s2/s3 *2^(32+β+1)
    M = M.astype(np.uint32)
    # exit()
    return M, N_REAL


In [4]:
import torch
M=torch.randint(1,10,[1,10])
for i,ii in enumerate(M):
    print(ii)
# for i, ii in enumerate(M):  # enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标
#     while not (ii >= 0.5 and ii <= 1.0):  # 左移到（0.5，1） 左移一次相当于*2
#         ii *= 2
#     pass
#     mmmm = ii * (2 ** 32)  # 乘2^32
#     SCALE[i] = mmmm.astype(np.int32)

tensor([5, 8, 3, 6, 9, 3, 2, 8, 6, 9])


In [22]:
print(torch.round(torch.tensor(2.5)))
torch.round(torch.tensor(1.5))

tensor(2.)


tensor(2.)

In [2]:
round(2.5)

2

In [1]:
#统计计算时间
import torch.nn as nn
import torch
import time
torch.cuda.is_available()

InChannel=8
OutChannel=768
FeatureSize=224
Device='cuda'
times=1000
net=nn.Conv2d(InChannel,OutChannel,16,bias=False,stride=16,padding=0)
input=torch.rand([1,InChannel,FeatureSize,FeatureSize])
net = net.to(Device)  # 网络模型
input = input.to(Device)  # 输入
start_event = torch.cuda.Event(enable_timing=True)
end_event = torch.cuda.Event(enable_timing=True)
# 记录开始事件
start_event.record()
# 执行卷积操作

for i in range(times):
    output = net(input)
# 记录结束事件并同步
end_event.record()
torch.cuda.synchronize()

# 计算执行时间（单位：毫秒）
elapsed_time = start_event.elapsed_time(end_event)

print(f"卷积操作在GPU上的执行时间为:{elapsed_time/times} 毫秒")

卷积操作在GPU上的执行时间为:4.78804248046875 毫秒


In [120]:
#接着上面
Device='cpu'
times=10000
net=nn.Conv2d(InChannel,OutChannel,16,bias=False,stride=16,padding=0)
input=torch.rand([1,InChannel,FeatureSize,FeatureSize])
net = net.to(Device)  # 网络模型
input = input.to(Device)  # 输入
time_start = time.time()
for i in range(times):
    output = net(input)
time_end = time.time()
time_sum = time_end - time_start
print(time_sum)

89.7195680141449


In [16]:
x1=torch.rand(197,768)
x2=torch.rand(197,768)
error=x1-x2
print(error.sum())
(x1-x2).sum()

tensor(-21.5111)


tensor(-21.5111)

In [18]:
import torch
import torch.nn.functional as F

# 创建两个输入张量
tensor1 = torch.randn(1, 197, 786)
tensor2 = torch.randn(1, 197, 786)

# 将张量移动到GPU上（如果可用）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tensor1 = tensor1.to(device)
tensor2 = tensor2.to(device)

# 将张量展平为二维矩阵
tensor1_flat = tensor1.view(1, -1)  # shape: [1, 155142]
tensor2_flat = tensor2.view(1, -1)  # shape: [1, 155142]

# 计算余弦相似度
similarity = F.cosine_similarity(tensor1_flat, tensor2_flat)

print("余弦距离:", similarity.item())

余弦距离:%d%d -0.0008845366537570953 -0.0008845366537570953
